In [10]:
#How many missing values exist in the dataset, and in which columns?
import pandas as pd
import numpy as np

file_path = "Electric_Vehicle_Population_Data.csv"

df = pd.read_csv(file_path)

print("📊 Dataset Overview:")
print(f"Total Rows   : {df.shape[0]}")
print(f"Total Columns: {df.shape[1]}")

df.head()


📊 Dataset Overview:
Total Rows   : 247344
Total Columns: 17


,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,1N4BZ0CP5G,King,Seattle,WA,98125.0,2016,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,46.0,349455557,POINT (-122.30253 47.72656),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
1,KNDJX3AEXG,King,Renton,WA,98058.0,2016,KIA,SOUL,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,93.0,31950.0,11.0,210641315,POINT (-122.08747 47.4466),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
2,5YJ3E1EB2J,King,Seattle,WA,98115.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215.0,0.0,43.0,171417494,POINT (-122.31676 47.68156),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,1C4RJXN64R,Kitsap,Bremerton,WA,98312.0,2024,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21.0,0.0,26.0,262542927,POINT (-122.65223 47.57192),PUGET SOUND ENERGY INC,5.303508e+10
4,5YJ3E1EB1J,Thurston,Olympia,WA,98512.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215.0,0.0,35.0,110432815,POINT (-122.9131 47.01359),PUGET SOUND ENERGY INC,5.306701e+10


In [11]:
#How should missing or zero values in the Base MSRP and Electric Range columns be handled?
import pandas as pd
import numpy as np

file_path = "Electric_Vehicle_Population_Data.csv"  # Update path if needed
df = pd.read_csv(file_path)

df['Base MSRP'] = pd.to_numeric(df['Base MSRP'], errors='coerce')
df['Electric Range'] = pd.to_numeric(df['Electric Range'], errors='coerce')

df['Base MSRP'] = df['Base MSRP'].replace(0, np.nan)
df['Electric Range'] = df['Electric Range'].replace(0, np.nan)

df['Base MSRP'] = df.groupby('Model')['Base MSRP'].transform(
    lambda x: x.fillna(x.median())
)

df['Electric Range'] = df.groupby('Model')['Electric Range'].transform(
    lambda x: x.fillna(x.median())
)

df.dropna(subset=['Base MSRP', 'Electric Range'], inplace=True)

print("✅ Cleaned Columns Summary:")
print(df[['Base MSRP', 'Electric Range']].describe())


✅ Cleaned Columns Summary:
           Base MSRP  Electric Range
count   17997.000000    17997.000000
mean    60087.764905      115.687392
std     19092.301267       98.822761
min     31950.000000       12.000000
25%     39995.000000       32.000000
50%     64950.000000       35.000000
75%     69900.000000      210.000000
max    845000.000000      337.000000


In [12]:
#Are there duplicate records in the dataset? If so, how should they be managed?

import pandas as pd

file_path = "Electric_Vehicle_Population_Data.csv"  # Update path if needed
df = pd.read_csv(file_path)

exact_duplicates = df[df.duplicated()]
print(f"🔍 Exact duplicate rows found: {len(exact_duplicates)}")

df = df.drop_duplicates()
print(f"✅ Dataset after removing exact duplicates: {len(df)} rows")

if 'VIN (1-10)' in df.columns:
    vin_duplicates = df[df.duplicated(subset=['VIN (1-10)'], keep=False)]
    print(f"🔍 Duplicate VIN entries found: {len(vin_duplicates)}")


    df = df.drop_duplicates(subset=['VIN (1-10)'], keep='last')
    print(f"✅ Dataset after keeping last record per VIN: {len(df)} rows")
else:
    print("⚠️ 'VIN (1-10)' column not found in the dataset.")


🔍 Exact duplicate rows found: 0
✅ Dataset after removing exact duplicates: 247344 rows
🔍 Duplicate VIN entries found: 244521
✅ Dataset after keeping last record per VIN: 14792 rows


In [13]:
#How can VINs be anonymized while maintaining uniqueness?

import pandas as pd
import hashlib

file_path = "Electric_Vehicle_Population_Data.csv"  # Update if needed
df = pd.read_csv(file_path)

if 'VIN (1-10)' in df.columns:
    
    def hash_vin(vin):
        return hashlib.sha256(str(vin).encode()).hexdigest()

    df['VIN_Anon'] = df['VIN (1-10)'].apply(hash_vin)

    df['VIN_ID'] = pd.factorize(df['VIN (1-10)'])[0]

    df.drop(columns=['VIN (1-10)'], inplace=True)

    print("🔐 VIN anonymization completed. Sample:")
    print(df[['VIN_Anon', 'VIN_ID']].head())

else:
    print("⚠️ Column 'VIN (1-10)' not found in dataset.")


🔐 VIN anonymization completed. Sample:
                                            VIN_Anon  VIN_ID
0  ba814b799f5bee2ad2c4b60bd963cb070cbda908e215cd...       0
1  ba258d18a33e17ab902eb1611576c93f3b5692f9f33f94...       1
2  2e9a11a90792ccfe7674388860f253e0288e53d8ec544e...       2
3  15768eb7a92d82af83f8ef28434832fd4f547490c7f524...       3
4  2984ac80c258388c54f6ddef953f580f6a76e54cbbd670...       4


In [13]:
#How can VINs be anonymized while maintaining uniqueness?

import pandas as pd
import hashlib

file_path = "Electric_Vehicle_Population_Data.csv"  # Update if needed
df = pd.read_csv(file_path)

if 'VIN (1-10)' in df.columns:
    
    def hash_vin(vin):
        return hashlib.sha256(str(vin).encode()).hexdigest()

    df['VIN_Anon'] = df['VIN (1-10)'].apply(hash_vin)

    df['VIN_ID'] = pd.factorize(df['VIN (1-10)'])[0]

    df.drop(columns=['VIN (1-10)'], inplace=True)

    print("🔐 VIN anonymization completed. Sample:")
    print(df[['VIN_Anon', 'VIN_ID']].head())

else:
    print("⚠️ Column 'VIN (1-10)' not found in dataset.")


🔐 VIN anonymization completed. Sample:
                                            VIN_Anon  VIN_ID
0  ba814b799f5bee2ad2c4b60bd963cb070cbda908e215cd...       0
1  ba258d18a33e17ab902eb1611576c93f3b5692f9f33f94...       1
2  2e9a11a90792ccfe7674388860f253e0288e53d8ec544e...       2
3  15768eb7a92d82af83f8ef28434832fd4f547490c7f524...       3
4  2984ac80c258388c54f6ddef953f580f6a76e54cbbd670...       4


In [15]:
#How can Vehicle Location (GPS coordinates) be cleaned or converted for better readability?

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

file_path = "Electric_Vehicle_Population_Data.csv"
df = pd.read_csv(file_path)

def extract_lat_lon(point_str):
    """
    Converts 'POINT (-122.12345 47.12345)' into [Latitude, Longitude]
    """
    try:
        lon, lat = point_str.replace("POINT (", "").replace(")", "").split()
        return pd.Series({"Latitude": float(lat), "Longitude": float(lon)})
    except:
        return pd.Series({"Latitude": np.nan, "Longitude": np.nan})


df[['Latitude', 'Longitude']] = df['Vehicle Location'].apply(extract_lat_lon)


df = df[
    df['Latitude'].between(-90, 90) &
    df['Longitude'].between(-180, 180)
]


geolocator = Nominatim(user_agent="ev_cleaner")

def get_address(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        return location.address if location else None
    except GeocoderTimedOut:
        return None
    except:
        return None

from tqdm.notebook import tqdm
tqdm.pandas()

df['Address'] = df.progress_apply(lambda row: get_address(row['Latitude'], row['Longitude']), axis=1)


df[['Latitude', 'Longitude', 'Address']].head()


ModuleNotFoundError: No module named 'geopy'